# Notes:
* Greg didnt get any failures, so no idea if the last few commands work or not.

# 1. About
This notebook runs through the Mothur MiSeq SOP tutorial found here: http://www.mothur.org/wiki/MiSeq_SOP .

# 2. Installation
This tutorial requires Qiime and PANDAseq, both of which can be downloaded/installed as directed below.  

### Qiime
http://qiime.org/install/install.html
Qiime suggest using their pre-packaged Virtual Machine image.  This is quite big (4GB), but comes with a ready-to-go configuration.

### PANDAseq
https://github.com/neufeld/pandaseq/wiki/Installation.
PANDAseq doesn't come with Qiime, so you'll need to install it separately.
NOTE: Because Qiime uses Ubuntu 12.04, you may have issues calling apt-get update.  In this case, do this instead:  

!sudo apt-add-repository ppa:neufeldlab/ppa && sudo apt-get install pandaseq

# 3. MothurMagic
To use Mothur in IPython, we need to load the mothurmagic extension. See https://github.com/SchlossLab/ipython-mothurmagic for more details.

In [1]:
%install_ext https://raw.githubusercontent.com/SchlossLab/ipython-mothurmagic/master/mothurmagic.py
%load_ext mothurmagic

/home/qiime/anaconda2/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed mothurmagic.py. To use it, type:
  %load_ext mothurmagic


# 3. Necessary Files
First, we should download the files used in the tutorial.  The below cell does just that.  In total, the zipped files are about 36MB and may take a few minutes to download/unizp.  See http://www.mothur.org/wiki/MiSeq_SOP#Logistics for more details.

In [2]:
#####################################################
##### Downloads and unzips all tutorial files #######
#####################################################

import urllib2
import zipfile
import os
from subprocess import call

# make a directory for our tutorial, and jump into it
root="qiime_notebook"
if os.getcwd().split('/')[-1] != root:
    if not os.path.isdir(root):
        os.mkdir(root)
    os.chdir(root)
print "Current working directory is: " + os.getcwd()

# Files to grab,''
files = [('https://raw.githubusercontent.com/edamame-course/2014-tutorials/master/misc/QIIMETutorial_Misc/pandaseq_merge.sh',''),
         ('http://www.mothur.org/w/images/d/d6/MiSeqSOPData.zip',''),
         ('https://raw.githubusercontent.com/edamame-course/2014-tutorials/master/misc/QIIMETutorial_Misc/Schloss_Map.txt','')]

# File Directory names:
seq='MiSeq_SOP'
    
# Grab and unzip the files
for url, path in files:
    target = url.split('/')[-1]
    if not os.path.isfile(target):
        resource = urllib2.urlopen(url)
        print "Downloading " + target + "...\n"
        open(target,'wb').write(resource.read())
        
        if url.split('.')[-1] == '.zip':
            print "Extracting " + target + "...\n"
            zipfile.ZipFile(target).extractall(path)
print "\nDone!"

Current working directory is: /home/qiime/Desktop/Qiime_Notebook/qiime_notebook

Done!


# 5. Commands

## 1. Get a list of Read IDs
We want to use PANDAseq to handle our paired-end merging, but PANDAseq require that you tell it explicitly which two files to merge.  Notice that the .fastq files in our directory are all named systematically: F3D[day]_S[id]_L001_[L/R]_001.fastq, where [day] is the day the sample was taken, [id] is a sample ID#, and [L/R] is either an L or R denoting a left/right read.  We can take advantage of this systematic naming with a bash script to generate the list of filenames we want to have PANDAseq run.  Our sample dataset includes a list of all the included .fastq files.  This list is called stability.files.  We can use grep to pull out the unique F3D[day]_S[id] combinations.  The following command below will read stability.files, pull out the F3D[day]_S[id] combinations and write them to a text file called SchlossSampleNames.txt.
### 1.1 Output Files
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>SchlossSampleNames.txt </td><td>A textfile where each line contains the  F3D[day]_S[id] identifier corresponding to file name pairs in stability.files</td></tr>
</table>


In [3]:
!cat MiSeq_SOP/stability.files | grep -oP "\t\K(F.*)(?=_L001_R1_001.fastq)" > SchlossSampleNames.txt

## 1.  PANDAseq - Assembles left and right reads.
### Options:
-f    Forward Read file in fastq format.
-r    Reverese Read file in fastq format.
-w    File location to write the joined read file.
-g    File location to write the log file.
-l    Max length for a joined read.  Reads longer than this length will be discarded.
-t    Minimum match threshold.  Reads that do not match above this threshold will be discarded.

### Usage: 
pandaseq -f forward_read.fastq -r reverse_read.fastq -w output_file_location -g log_file_location -L length -t minimum_match_%

Notice that this one command will merge a pair of files into one long read.  To join 1,000
files, we would need 1,000 commands.  Since we don't want to type those commands manually, 
we should definitely automate it.  The bash script below does just that by reading  SchlossSampleNames.txt, automatically generating the forward, reverse, and output file names, and calling PANDAseq.

### 1.1 Output Files:
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>pandaseq_merged_reads/F3D0_S188.fasta</td><td>Combined read for F3D0_S188.</td></tr>
<tr><td>pandaseq_merged_reads/F3D0_S188.log</td><td>Log for F3D0_S188 with quality info and warnings?.</td></tr>
<tr><td>pandaseq_merged_reads/F3D1_S189.fasta</td><td>Combined read for F3D1_S189.</td></tr>
<tr><td>pandaseq_merged_reads/F3D1_S189.log</td><td>Log for F3D1_S189 with quality info and warnings?.</td></tr>
<tr><td>...more logs and fastas...</td><td>...</td></tr><tr>
<td>pandaseq_merged_reads/F3D150_S216.fasta</td><td>Combined read for F3D150_S216.</td></tr>
<tr><td>pandaseq_merged_reads/F3D150_S216.log</td><td>Log for F3D150_S216 with quality info and warnings?.</td></tr>
</table>

In [4]:
!chmod +x pandaseq_merge.sh
!mkdir pandaseq_merged_reads
!./pandaseq_merge.sh

## Aggregated Assembled Reads
QIIME expects all of the data to be in one file, and, currently, we have one separate fastq file for each assembled read. We will add labels to each sample and merge into one fasta using the add_qiime_labels.py script. 
### Documentation
http://qiime.org/scripts/add_qiime_labels.html
### Output Files
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>combined_fasta/combined_seqs.fna</td><td>Contains all assembled reads, labeled with sample info.</td></tr>
</table>

In [5]:
!add_qiime_labels.py -i pandaseq_merged_reads/ -m Schloss_Map.txt -c InputFileName -n 1 -o combined_fasta

## Count the number of Sequences
Counts the number of sequences in a fasta file.
### Documentation
http://qiime.org/scripts/count_seqs.html

In [6]:
!count_seqs.py -i combined_fasta/combined_seqs.fna


105904  : combined_fasta/combined_seqs.fna (Sequence lengths (mean +/- std): 252.5030 +/- 0.5343)
105904  : Total


## Summarize the fasta file
Summarizes a fasta file.
### Documentation
http://www.mothur.org/wiki/Summary.seqs
### Output Files	
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>combined_seqs.fna.summary</td><td>Table summarizing the fasta file (start, end, nbases, ambigs, homopolymer, numSeqs).</td></tr>
</table>

In [7]:
%%mothur
%%summary.seqs(fasta=combined_fasta/combined_seqs.fna)

mothur > %%summary.seqs(fasta=combined_fasta/combined_seqs.fna)

mothur > quit()


## Pick OTUS
Runs denovo clustering over the combined reads.
### Documentation
http://qiime.org/scripts/pick_otus.html
### Output Files	
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>cdhit_picked_otus/combined_seqs_otus.log</td><td>Dump of denovo clustering parameters.</td></tr>
<tr><td>cdhit_picked_otus/combined_seqs_otus.txt</td><td>Lists clusters and the sequences assigned to them.</td></tr>

</table>

In [8]:
!pick_otus.py -i combined_fasta/combined_seqs.fna -m cdhit -o cdhit_picked_otus/ -s 0.97 -n 100

## Pick OTUS
Picks a representative sequence from each cluster.
### Documentation
http://qiime.org/scripts/pick_rep_set.html
### Output Files	
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>cdhit_rep_seqs/cdhit_rep_seqs.fasta</td><td>List of representative sequences.</td></tr>
<tr><td>cdhit_rep_seqs/cdhit_rep_seqs.log</td><td>Dump of parameters.</td></tr>


</table>

In [9]:
!mkdir cdhit_rep_seqs/
!pick_rep_set.py -i cdhit_picked_otus/combined_seqs_otus.txt -f combined_fasta/combined_seqs.fna -o cdhit_rep_seqs/cdhit_rep_seqs.fasta -l cdhit_rep_seqs/cdhit_rep_seqs.log

## Count the number of Sequences
Counts the number of sequences in a fasta file.
### Documentation
http://qiime.org/scripts/count_seqs.html

In [10]:
!count_seqs.py -i cdhit_rep_seqs/cdhit_rep_seqs.fasta


689  : cdhit_rep_seqs/cdhit_rep_seqs.fasta (Sequence lengths (mean +/- std): 252.7649 +/- 0.5440)
689  : Total


## Align representative sequences
Uses PYNAST to align representative sequences against the greengenes database.
### Documentation
http://qiime.org/scripts/pick_rep_set.html
### Output Files	
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>pynast_aligned/cdhit_rep_seqs_aligned.fasta</td><td>The aligned sequences.</td></tr>
<tr><td>pynast_aligned/cdhit_rep_seqs_failures.fasta</td><td>Sequences that failed to align.</td></tr>
<tr><td>pynast_aligned/cdhit_rep_seqs_failures_names.txt</td><td>List of representative sequences.</td></tr>
<tr><td>pynast_aligned/cdhit_rep_seqs_log.txt</td><td>Table on alignment data.  Lists sequence, match ID, BLAST % match, etc.</td></tr>

</table>

In [11]:
!align_seqs.py -i cdhit_rep_seqs/cdhit_rep_seqs.fasta -o pynast_aligned/ -e 100

## Count the number of Successes and Failures
Counts the number of sequences in a fasta file.
### Documentation
http://qiime.org/scripts/count_seqs.html

In [12]:
!count_seqs.py -i pynast_aligned/cdhit_rep_seqs_failures.fasta
!count_seqs.py -i pynast_aligned/cdhit_rep_seqs_aligned.fasta


0  : pynast_aligned/cdhit_rep_seqs_failures.fasta
0  : Total

689  : pynast_aligned/cdhit_rep_seqs_aligned.fasta (Sequence lengths (mean +/- std): 7682.0000 +/- 0.0000)
689  : Total


## Generate a List of Bad OTUs
Use Grep to generate a list of OTUs that failed to align.
### Documentation
http://unixhelp.ed.ac.uk/CGI/man-cgi?grep
### Output Files	
<table>
<hr><td>Output File</td><td>Description</td></hr>
<tr><td>pynast_aligned/cdhit_rep_seqs_failures_names.txt</td><td>Names of filtered </td></tr>
</table>

In [13]:
!grep -o -E "^>\w+" pynast_aligned/cdhit_rep_seqs_failures.fasta | tr -d ">" > pynast_aligned/cdhit_rep_seqs_failures_names.txt